# Daily

In [ ]:
import pandas as pd
import numpy as np
from autots import AutoTS
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Load data
df = pd.read_csv("between_365_300_data.csv", parse_dates=["date"])

# Convert date column to datetime
df["date"] = pd.to_datetime(df["date"])

# Replace 'user_id_value' with the actual user ID you want to filter
user_id_value = "dfc84fb2db544292a7083a9fdbb211a7eb4a0ab330436a31b9199bf3835d1278"  # Example user ID

# Filter data for the specific user
user_data = df[df["profile_id"] == user_id_value]

# Extract year, month, and day
user_data["year"] = user_data["date"].dt.year
user_data["month"] = user_data["date"].dt.month
user_data["day"] = user_data["date"].dt.day

# Aggregate: Keep last follower count per day for each profile
daily_df = (
    user_data.sort_values(["profile_id", "date"])
    .groupby(["profile_id", "year", "month", "day"])
    .last()
    .reset_index()
)

# Convert back to date format
daily_df["date"] = pd.to_datetime(daily_df[["year", "month", "day"]])

# Drop unnecessary columns
daily_df = daily_df[["date", "profile_id", "followers_num"]]

# Fill missing values
daily_df.fillna(method='ffill', inplace=True)
daily_df.fillna(method='bfill', inplace=True)
daily_df.fillna(0, inplace=True)

# Sort data by date to ensure correct splitting
daily_df = daily_df.sort_values("date")

# Find the total number of unique days
unique_dates = daily_df["date"].unique()

# Determine the split index (75% train-test split)
split_index = int(len(unique_dates) * 0.75)

# Get the corresponding date for the split
split_date = unique_dates[split_index]  # This is the cutoff date

# Perform the split dynamically
train_df = daily_df[daily_df["date"] < split_date]
test_df = daily_df[daily_df["date"] >= split_date]

# # IF you need to verify the split, uncomment the code below
# print("Train Data Range:", train_df["date"].min(), "to", train_df["date"].max())
# print("Test Data Range:", test_df["date"].min(), "to", test_df["date"].max())
# print(f"Split Date: {split_date}, Total Unique Days: {len(unique_dates)}")


# Pivot and convert dates to daily format
def prepare_for_autts(df):
    df = df.pivot(index='date', columns='profile_id', values='followers_num')
    df = df.fillna(method='ffill')
    df.index = df.index.to_period('D').to_timestamp('D')  # Convert to daily timestamp
    df = df.fillna(method='ffill')
    return df

train_df = prepare_for_autts(train_df)
test_df = prepare_for_autts(test_df)

# Ensure no missing values
train_df.fillna(method='ffill', inplace=True)
train_df.fillna(method='bfill', inplace=True)
train_df.fillna(0, inplace=True)

test_df.fillna(method='ffill', inplace=True)
test_df.fillna(method='bfill', inplace=True)
test_df.fillna(0, inplace=True)

# ----- AUTOTS MODEL -----
# Initialize and train the AutoTS model for daily forecasting
model = AutoTS(
    forecast_length=len(test_df),  
    frequency='D',  # Daily frequency
    ensemble='simple',
    max_generations=10,
    num_validations=3,
    validation_method='backwards'
)

# Train the model
model = model.fit(train_df)

# Generate predictions
predictions = model.predict(prediction_interval=0.95, fail_on_forecast_nan=False)
forecast_df = predictions.forecast.fillna(0)

# Align test data with predictions
common_index = forecast_df.index.intersection(test_df.index)
forecast_df = forecast_df.loc[common_index]
test_df_aligned = test_df.loc[common_index]
test_df_aligned = test_df.loc[forecast_df.index]

# Evaluate AutoTS model
evaluation_results = []
for profile_id in test_df_aligned.columns:
    true_values = test_df_aligned[profile_id]
    predicted_values = forecast_df[profile_id]
    mae_autots = mean_absolute_error(true_values, predicted_values)
    mse_autots = mean_squared_error(true_values, predicted_values)
    evaluation_results.append({'Profile': profile_id, 'MAE_AutoTS_Daily': mae_autots, 'MSE_AutoTS_Daily': mse_autots})

# Create a comparison DataFrame
evaluation_df = pd.DataFrame(evaluation_results)

Train Data Range: 2023-01-01 00:00:00 to 2023-08-22 00:00:00
Test Data Range: 2023-08-23 00:00:00 to 2023-11-08 00:00:00
Split Date: 2023-08-23 00:00:00, Total Unique Days: 312
Using 4 cpus for n_jobs.
Data frequency is: D, used frequency is: D
Too many training validations for length of data provided, decreasing num_validations to 1
Model Number: 1 with model AverageValueNaive in generation 0 of 10
Model Number: 2 with model AverageValueNaive in generation 0 of 10
Model Number: 3 with model AverageValueNaive in generation 0 of 10


/tmp/ipykernel_5304/1116074175.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_data["year"] = user_data["date"].dt.year
/tmp/ipykernel_5304/1116074175.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_data["month"] = user_data["date"].dt.month
/tmp/ipykernel_5304/1116074175.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org

Model Number: 4 with model DatepartRegression in generation 0 of 10
Model Number: 5 with model DatepartRegression in generation 0 of 10
Model Number: 6 with model DatepartRegression in generation 0 of 10


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Model Number: 7 with model DatepartRegression in generation 0 of 10
Template Eval Error: ImportError('Tensorflow not available, install with pip install tensorflow.') in model 7 in generation 0: DatepartRegression
Model Number: 8 with model ETS in generation 0 of 10
Model Number: 9 with model ETS in generation 0 of 10
Model Number: 10 with model GLM in generation 0 of 10
Model Number: 11 with model GLM in generation 0 of 10
Model Number: 12 with model GLS in generation 0 of 10
Model Number: 13 with model GLS in generation 0 of 10
Model Number: 14 with model LastValueNaive in generation 0 of 10
Model Number: 15 with model LastValueNaive in generation 0 of 10
Model Number: 16 with model LastValueNaive in generation 0 of 10
Model Number: 17 with model LastValueNaive in generation 0 of 10
Model Number: 18 with model SeasonalNaive in generation 0 of 10
Model Number: 19 with model SeasonalNaive in generation 0 of 10
Model Number: 20 with model SeasonalNaive in generation 0 of 10
Model Number

...

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=1.51103e-24): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


prediction too long for indepedent=False, falling back on indepedent=True
85 - Ensemble with avg smape 10.47: 
86 - Ensemble with avg smape 10.47: 
87 - Ensemble with avg smape 10.48: 
88 - Ensemble with avg smape 10.46: 
89 - Ensemble with avg smape 10.75: 
90 - Ensemble with avg smape 10.84: 
91 - Ensemble with avg smape 10.71: 
92 - Ensemble with avg smape 10.71: 
93 - Ensemble with avg smape 10.77: 
94 - Ensemble with avg smape 10.73: 
95 - Ensemble with avg smape 10.47: 
96 - Ensemble with avg smape 10.47: 
97 - Ensemble with avg smape 19.72: 
98 - Ensemble with avg smape 10.48: 
Model Number: 99 of 8 with model Ensemble for Validation 1


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=1.51103e-24): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


prediction too long for indepedent=False, falling back on indepedent=True
99 - Ensemble with avg smape 10.47: 
100 - Ensemble with avg smape 10.47: 
101 - Ensemble with avg smape 10.48: 
102 - Ensemble with avg smape 10.46: 
103 - Ensemble with avg smape 10.75: 
104 - Ensemble with avg smape 10.84: 
105 - Ensemble with avg smape 10.71: 
106 - Ensemble with avg smape 10.71: 
107 - Ensemble with avg smape 10.77: 
108 - Ensemble with avg smape 10.73: 
109 - Ensemble with avg smape 10.47: 
110 - Ensemble with avg smape 10.47: 
111 - Ensemble with avg smape 19.72: 
112 - Ensemble with avg smape 10.48: 


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/numpy/core/_methods.py:176: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/numpy/core/_methods.py:187: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [ ]:
# Extract the predicted follower numbers for the specific user
user_predictions = forecast_df[[user_id_value]].reset_index()

# Rename columns for clarity
user_predictions.columns = ["date", "predicted_followers_num"]

# Display the pridiction table on daily basis
print(user_predictions)  # For console output
user_predictions.to_csv("user_predictions.csv", index=False)  # Save as CSV if needed

         date  predicted_followers_num
0  2023-08-23            618826.248022
1  2023-08-24            618826.248022
2  2023-08-25            618824.784733
3  2023-08-26            618821.861232
4  2023-08-27            618817.483658
..        ...                      ...
73 2023-11-04            617147.616509
74 2023-11-05            617141.973153
75 2023-11-06            617137.730736
76 2023-11-07            617134.897705
77 2023-11-08            617133.479749

[78 rows x 2 columns]


In [ ]:
# Check the evaluation results of MAE and MSE if needed
evaluation_results

[{'Profile': 'dfc84fb2db544292a7083a9fdbb211a7eb4a0ab330436a31b9199bf3835d1278',
  'MAE_AutoTS_Daily': 92096.98847030698,
  'MSE_AutoTS_Daily': 49052012572.104576}]